<a href="https://colab.research.google.com/github/AaravWattal/RRAM-Models/blob/main/CleanedConductanceAnalysisModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Datasets

In [77]:
from datascience import *
import pandas as pd
import math
import random
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from time import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor

In [78]:
# get file set_sweep for conductange range data
names = ["addr", "pw", "vwl", "vbsl", "preread", "postread"]
data = pd.read_csv("/content/set_sweep.csv", names=names)
data.head(20)

,addr,pw,vwl,vbsl,preread,postread
0,0,2.000000e-08,0.5,0.5,151240.470700,115068.986900
1,2,2.000000e-08,0.5,1.0,123855.302200,124977.565700
2,4,2.000000e-08,0.5,1.5,127236.989700,106676.196700
3,6,2.000000e-08,0.5,2.0,93031.982770,133852.645200
4,8,2.000000e-08,0.5,2.5,75606.517090,57549.247700
5,10,2.000000e-08,0.5,3.0,97572.302250,92171.401490
6,12,2.000000e-08,0.6,0.5,46314.963040,50373.876070
7,14,2.000000e-08,0.6,1.0,100303.315400,149505.440600
8,16,2.000000e-08,0.6,1.5,212694.720700,75614.854910
9,18,2.000000e-08,0.6,2.0,5747.716015,5608.268993


#Data Splitting and Organizing + Scaled Inputs and Outputs

In [79]:
x1 = data[['pw']]
x2 = data[['vwl']]
x3 = data[['vbsl']]
x4 = data[['preread']]
y = data[['postread']]

In [80]:
#y1 is postread conductance
y1 = [1/i for i in y["postread"]]

In [82]:
#I will take the max conductance value and map it to 1 by multiplying by 0.511*10^4
value = 0.511*10**4
print(max(y1)*value)

y3 = [i*value for i in y1]
print(y3)
print(max(y3))
print(min(y3))

1.0007432384902553
[0.04440814278169334, 0.04088733823049652, 0.0479019702433767, 0.03817630942119028, 0.08879351519307523, 0.055440189878792304, 0.10144146924288887, 0.03417935815240158, 0.0675793136954655, 0.9111545837722981, 0.07495584021412594, 0.1243437954209893, 0.03885021323551558, 0.07420944518300189, 0.14283555272626744, 0.10529447485095887, 0.09162448123142143, 0.9280431824224492, 0.03401659333416961, 0.03312149014048232, 0.0752245720810912, 0.38030933970218356, 0.28789993428517796, 0.8801065939637305, 0.07138916577659593, 0.03257415786404181, 0.0269493046451815, 0.35702739067189687, 0.468824987914591, 0.5525578349850624, 0.06464393065856265, 0.06276001550417036, 0.3604382166927755, 0.4708953677044949, 0.5558711069878407, 0.4989046182981523, 0.10397222341235829, 0.04388170323040603, 0.48811814314486185, 0.5192787196430975, 0.5116887319074322, 0.5486938406903997, 0.06573144276053425, 0.33055420906285843, 0.4765589671627595, 0.46629303266009114, 0.5958163938054075, 0.6640364883

###Now the inputs must be scaled one by one to 0-1.

####Pulse Width Scaling

In [89]:
#scale by mapping max pulse width to 1, multiply by 500000
value = 500000
print(max(x1['pw'])*value)
x1a = [i*value for i in x1['pw']]
print(x1a)

1.0
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,

####Wordline Voltage Scaling

In [90]:
#scale by mapping max vwl to 1, multiply by 0.2941176
value = 0.2941176
print(max(x2['vwl'])*value)
x2a = [i*value for i in x2['vwl']]
print(x2a)

0.9999998399999999
[0.1470588, 0.1470588, 0.1470588, 0.1470588, 0.1470588, 0.1470588, 0.17647055999999997, 0.17647055999999997, 0.17647055999999997, 0.17647055999999997, 0.17647055999999997, 0.17647055999999997, 0.20588231999999998, 0.20588231999999998, 0.20588231999999998, 0.20588231999999998, 0.20588231999999998, 0.20588231999999998, 0.2352940799999997, 0.2352940799999997, 0.2352940799999997, 0.2352940799999997, 0.2352940799999997, 0.2352940799999997, 0.2647058399999997, 0.2647058399999997, 0.2647058399999997, 0.2647058399999997, 0.2647058399999997, 0.2647058399999997, 0.2941175999999997, 0.2941175999999997, 0.2941175999999997, 0.2941175999999997, 0.2941175999999997, 0.2941175999999997, 0.32352936, 0.32352936, 0.32352936, 0.32352936, 0.32352936, 0.32352936, 0.35294111999999994, 0.35294111999999994, 0.35294111999999994, 0.35294111999999994, 0.35294111999999994, 0.35294111999999994, 0.38235288, 0.38235288, 0.38235288, 0.38235288, 0.38235288, 0.38235288, 0.41176463999999996, 0.411764639

####Bitline Voltage Scaling

In [92]:
#scale by mapping max vbsl to 1, multiply by 0.33333
value = 1/3
print(max(x3['vbsl'])*value)
x3a = [i*value for i in x3['vbsl']]
print(x3a)

1.0
[0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333, 1.0, 0.16666666666666666, 0.3333333333333333, 0.5, 0.6666666666666666, 0.8333333333333333,

####Preread Resistance/Conductance Scaling

In [93]:
#preread resistance to preread conductance
x5 = [1/i for i in x4["preread"]]

#scale by mapping max preread conductance to 1, multiply by 5190.251624
value = 5190.251624
print(max(x5)*value)
x5a = [i*value for i in x5]
print(x5a)

Min: 2.702846255980851e-07
Max: 0.00019266888629752488
0.9999999999999999
[0.03431787536746935, 0.0419057685202596, 0.040792002673417545, 0.05578997103428069, 0.06864820419940337, 0.053193903436874176, 0.11206425058608878, 0.051745563975645024, 0.024402352850688317, 0.9030111457237681, 0.052220750737850326, 0.04503025002890039, 0.03756509198429345, 0.061764905997106705, 0.0670966849598258, 0.03308572911042571, 0.04129617448821218, 0.9321091097918408, 0.050689766813140684, 0.03903911388690877, 0.051451425029408325, 0.03141834368221965, 0.05028259472042638, 0.8920189128175338, 0.07544311656360803, 0.05629576873173315, 0.040430838545868625, 0.07763444281700345, 0.051692768438176794, 0.059732115804074636, 0.06584071056366093, 0.032732674201760285, 0.02915856387354128, 0.058916226182580855, 0.05837993204966318, 0.05775307483666812, 0.11081906255861602, 0.06345831452609434, 0.07358622252453771, 0.05137600777109673, 0.05001116639318968, 0.05582771524428817, 0.04067913586220262, 0.070821197742

####Postread Resistance/Conductance Scaling

In [95]:
#scale by mapping max preread conductance to 1, multiply by 5106.204872
value = 5106.204872
print(max(y1)*value)
y1a = [i*value for i in y1]
print(y1a)

1.0
[0.04437516145368966, 0.04085697175649182, 0.04786639409689416, 0.038147956391675404, 0.08872756944831443, 0.05539901519837463, 0.10136613003343978, 0.03415397360462346, 0.067529123451703, 0.9104778815661919, 0.07490017152372277, 0.12425144696313638, 0.03882135970673749, 0.07415433083010983, 0.14272947068995687, 0.10521627406627156, 0.09155643296641032, 0.9273539372821713, 0.03399132966962419, 0.03309689125699233, 0.07516870380715912, 0.3800268890908792, 0.2876861148914786, 0.8794529506804162, 0.07133614600713696, 0.03254996547684294, 0.026929289760516242, 0.35676223124976264, 0.46847679792658037, 0.5521474577421718, 0.06459592048414536, 0.06271340448809985, 0.36017052409621175, 0.4705456400733705, 0.5554582690225726, 0.4985340885748191, 0.10389500464596016, 0.04384911288194861, 0.487755624390976, 0.5188930583498056, 0.5113087075955445, 0.5482863331838965, 0.06568262490556342, 0.3303087109152396, 0.4762050332527927, 0.4659467231210592, 0.5953738890149985, 0.6635433173752046, 0.0562

###Combining Data and Train Test Split

In [27]:
#Combine lists into dataframe
dataframe = pd.DataFrame(
    {'pw': x1a,
     'vwl': x2a,
     'vbsl': x3a,
     'preread': x5a
    })

In [28]:
#Train-Test 70%-30% split
x = dataframe[["pw","vwl","vbsl","preread"]].to_numpy()
y = np.array(y1a)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [29]:
X_entire_train = X_train
y_entire_train = y_train

###Training/Testing Model

In [96]:
print("Training MLPRegressor...")
tic = time()
est = make_pipeline(
    QuantileTransformer(),
    MLPRegressor(
        hidden_layer_sizes=(45, 23),
        learning_rate_init=0.03,
        early_stopping=True,
        random_state=0,
    ),
)
est.fit(X_entire_train, y_entire_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.5f}")

Training MLPRegressor...
done in 6.228s
Test R2 score: 0.96909


In [100]:
#Mean Squared Error
predictions = est.predict(X_train)
mean_squared_error(y_train, predictions, squared=False)

0.04944659853348425

In [101]:
mean_squared_error(y_train, predictions, squared=True)

0.002444966106531567

In [102]:
#Mean Squared Error
predictions = est.predict(X_test)
mean_squared_error(y_test, predictions, squared=False)

0.04620296351725698

#Various Subsets

In [103]:
total_dataframe = pd.DataFrame(
    {'pw': x1a,
     'vwl': x2a,
     'vbsl': x3a,
     'preread': x5a,
     'postread': y1a
    })

###Postread Conductance ONLY Subset

In [37]:
cond_subset = []

count = 0
for element in total_dataframe['postread']:
  if element >= 0.5:
    cond_subset.append(element)
    count += 1

print(count/(len(total_dataframe['postread'])))
# the above code tells us around 56% of the postread conductances are above the 0.5

0.557098388671875


In [39]:
cond_df = total_dataframe[total_dataframe['postread'] >= 0.5]
cond_df

,pw,vwl,vbsl,preread,postread
9,0.01,0.176471,0.666667,0.903011,0.910478
17,0.01,0.205882,1.000000,0.932109,0.927354
23,0.01,0.235294,1.000000,0.892019,0.879453
29,0.01,0.264706,1.000000,0.059732,0.552147
34,0.01,0.294118,0.833333,0.058380,0.555458
...,...,...,...,...,...
32753,1.00,0.970588,1.000000,0.089709,0.852628
32756,1.00,1.000000,0.500000,0.042824,0.635402
32757,1.00,1.000000,0.666667,0.108055,0.750463
32758,1.00,1.000000,0.833333,0.028018,0.793074


In [108]:
#Note: look into finding way to use dataframe operations instead of this sketchy list stuff
x11 = cond_df[['pw']]
x22 = cond_df[['vwl']]
x33 = cond_df[['vbsl']]
x44 = cond_df[['preread']]
y11 = cond_df[['postread']]

x1b = list(x11['pw'])
x2b = list(x22['vwl'])
x3b = list(x33['vbsl'])
x4b = list(x44['preread'])
y1b = list(y11['postread'])

In [42]:
#Combine lists into dataframe
dataframe2 = pd.DataFrame(
    {'pw': x1b,
     'vwl': x2b,
     'vbsl': x3b,
     'preread': x4b
    })

In [43]:
#Train-Test 70%-30% split
x = dataframe2[["pw","vwl","vbsl","preread"]].to_numpy()
y = np.array(y1b)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [104]:
est.fit(X_entire_train, y_entire_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.5f}")

done in 369.022s
Test R2 score: 0.96909


In [139]:
#Mean Squared Error
predictions = est.predict(X_entire_train)
mean_squared_error(y_entire_train, predictions, squared=False)

0.062183079317664716

###(Significant Change 50%) Preread - Postread Conductance Subset

In [109]:
subtr_df = total_dataframe[np.abs(total_dataframe['preread'] - total_dataframe['postread']) >= 0.5]
subtr_df

,pw,vwl,vbsl,preread,postread
46,0.01,0.352941,0.833333,0.052877,0.595374
47,0.01,0.352941,1.000000,0.055367,0.663543
51,0.01,0.382353,0.666667,0.036685,0.607948
52,0.01,0.382353,0.833333,0.062256,0.712036
56,0.01,0.411765,0.500000,0.032252,0.578200
...,...,...,...,...,...
32753,1.00,0.970588,1.000000,0.089709,0.852628
32756,1.00,1.000000,0.500000,0.042824,0.635402
32757,1.00,1.000000,0.666667,0.108055,0.750463
32758,1.00,1.000000,0.833333,0.028018,0.793074


In [110]:
#44% of the data has a different of over 0.5 between preread and postread resistance
len(subtr_df['postread']) / len(total_dataframe['postread'])

0.43646240234375

In [50]:
x111 = subtr_df[['pw']]
x222 = subtr_df[['vwl']]
x333 = subtr_df[['vbsl']]
x444 = subtr_df[['preread']]
y111 = subtr_df[['postread']]

x1c = list(x111['pw'])
x2c = list(x222['vwl'])
x3c = list(x333['vbsl'])
x4c = list(x444['preread'])
y1c = list(y111['postread'])

In [52]:
#Combine lists into dataframe
dataframe2 = pd.DataFrame(
    {'pw': x1c,
     'vwl': x2c,
     'vbsl': x3c,
     'preread': x4c
    })

In [53]:
#Train-Test 70%-30% split
x = dataframe2[["pw","vwl","vbsl","preread"]].to_numpy()
y = np.array(y1c)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [111]:
est.fit(X_entire_train, y_entire_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.5f}")

done in 463.580s
Test R2 score: 0.96909


In [125]:
#Mean Squared Error
predictions = est.predict(X_entire_train)
mean_squared_error(y_entire_train, predictions, squared=False)

0.062183079317664716

In [126]:
#Mean Squared Error
predictions = est.predict(X_test)
mean_squared_error(y_test, predictions, squared=False)

0.04620296351725698

In [127]:
#relatively similar mean squared error to the entire dataset
mean_squared_error(y_test, predictions, squared=True)

0.002134713837776979

###(Miniscule Change 5%) Preread - Postread Conductance Subset

In [61]:
ten_df = total_dataframe[np.abs(total_dataframe['preread'] - total_dataframe['postread']) <= 0.05]
ten_df

,pw,vwl,vbsl,preread,postread
0,0.01,0.147059,0.166667,0.034318,0.044375
1,0.01,0.147059,0.333333,0.041906,0.040857
2,0.01,0.147059,0.500000,0.040792,0.047866
3,0.01,0.147059,0.666667,0.055790,0.038148
4,0.01,0.147059,0.833333,0.068648,0.088728
...,...,...,...,...,...
32763,0.01,0.147059,0.666667,0.047479,0.054263
32764,0.01,0.147059,0.833333,0.045670,0.045872
32765,0.01,0.147059,1.000000,0.055141,0.071448
32766,0.01,0.176471,0.166667,0.079956,0.066338


In [62]:
#62% of the data has a difference of over 0.05 between preread and postread resistance
#this is the data population where the pulse width likely actually affects the resistance
len(ten_df['postread']) / len(total_dataframe['postread'])

0.379150390625

In [115]:
x1111 = ten_df[['pw']]
x2222 = ten_df[['vwl']]
x3333 = ten_df[['vbsl']]
x4444 = ten_df[['preread']]
y1111 = ten_df[['postread']]

x1d = list(x1111['pw'])
x2d = list(x2222['vwl'])
x3d = list(x3333['vbsl'])
x4d = list(x4444['preread'])
y1d = list(y1111['postread'])

In [116]:
#Combine lists into dataframe
dataframe2 = pd.DataFrame(
    {'pw': x1d,
     'vwl': x2d,
     'vbsl': x3d,
     'preread': x4d
    })

In [117]:
#Train-Test 70%-30% split
x = dataframe2[["pw","vwl","vbsl","preread"]].to_numpy()
y = np.array(y1d)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [119]:
est.fit(X_entire_train, y_entire_train)
print(f"done in {time() - tic:.3f}s")
print(f"Test R2 score: {est.score(X_test, y_test):.5f}")

done in 10.726s
Test R2 score: 0.96909


In [122]:
#Mean Squared Error
predictions = est.predict(X_entire_train)
mean_squared_error(y_entire_train, predictions, squared=False)

0.062183079317664716

In [123]:
#Mean Squared Error
predictions = est.predict(X_test)
mean_squared_error(y_test, predictions, squared=False)

0.04620296351725698

In [124]:
#relatively similar mean squared error to the entire dataset
mean_squared_error(y_test, predictions, squared=True)

0.002134713837776979